In [1]:
#libraries
'''
!pip install transformers==4.31.0 
!pip install sentence-transformers==2.2.2 
!pip accelerate==0.21.0 \
!pip xformers==0.0.20 \
'''

'\n!pip install transformers==4.31.0 \n!pip install sentence-transformers==2.2.2 \n!pip accelerate==0.21.0 !pip xformers==0.0.20 '

In [2]:
# execution time tracking
#!pip install ipython-autotime
%load_ext autotime

time: 0 ns (started: 2025-11-12 14:45:25 +01:00)


In [3]:
#specific version to avoid error with vector store
#!pip install chromadb==0.5.3
#!pip install datasets

time: 0 ns (started: 2025-11-12 14:45:44 +01:00)


In [4]:
#loading training data for providing context to the model
import pandas as pd
train_data_df=pd.read_csv("C:/Users/USER/Documents/Projects/Instruction Dataset/LLM Training Data/new data/train_data_split.csv")
## Convert the DataFrame to a format compatible with Hugging Face Datasets
train_data_df['Instruction'] = train_data_df['Instruction']+ ". Input Text: " + train_data_df['input_text'] + "\n\nExtracted Properties:\n\n"+ train_data_df['Output'].astype(str)
train_dataset_dict = {
    'instruction': train_data_df['Instruction'].tolist()
}

C:\Users\USER\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


time: 1.73 s (started: 2025-11-12 14:45:44 +01:00)


In [5]:
#verify
#train_dataset_dict

time: 0 ns (started: 2025-11-12 14:45:46 +01:00)


In [6]:
#creating context document from the list
docs=train_dataset_dict['instruction']

time: 0 ns (started: 2025-11-12 14:45:48 +01:00)


In [7]:
#initializing the embedding model
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import torch
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', cache_folder = '/data/base_models')

time: 23.9 s (started: 2025-11-12 14:45:49 +01:00)


C:\Users\USER\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
#do the embedding
embeddings = embedding_model.encode(docs)

time: 3min 56s (started: 2025-11-12 14:46:13 +01:00)


In [9]:
#verify the embeddings
embeddings.shape

(1040, 768)

time: 0 ns (started: 2025-11-12 14:50:09 +01:00)


In [10]:
#indexing-run this only once
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="q-rag_GPT4-Turbo")

time: 672 ms (started: 2025-11-12 14:50:09 +01:00)


In [11]:
#addding the train data
collection.add(
    embeddings = embeddings,
    documents=docs,
    ids= [str(i) for i in range(len(docs))]
)

time: 2.88 s (started: 2025-11-12 14:50:10 +01:00)


In [12]:
#retrieving
def retrieve_vector_db(query, n_results=3):
    results = collection.query(
    query_embeddings = embedding_model.encode(query).tolist(),
    n_results=n_results
    )
    return results['documents']

time: 16 ms (started: 2025-11-12 14:50:13 +01:00)


In [13]:
#loading test data for model evaluation
import pandas as pd
test_data_path="C:/Users/USER/Documents/Projects/Instruction Dataset/LLM Training Data/new data/test_data_split.csv"

time: 0 ns (started: 2025-11-12 14:50:13 +01:00)


In [14]:
#processing the test data to add the queries
#defining the queries for the properties-we need to choose a query based on the input
query_temperature="working temperature of the quantum cascade laser"
query_power= "optical power of the quantum cascade laser"
query_frequency="lasing frequency of the quantum cascade laser"
query_hs="heterostructure of the quantum cascade laser"
query_design_type="design type of the quantum cascade laser"

time: 0 ns (started: 2025-11-12 14:50:13 +01:00)


In [15]:
#checking for the query keywords
def check_text(row, column_name):
    text = row[column_name].lower()
    if "working temperature" in text:
        return query_temperature
    elif "optical power" in text:
        return query_power
    elif "lasing frequency" in text:
        return query_frequency
    elif "query_hs" in text:
        return query_hs
    else:
        return query_design_type

time: 0 ns (started: 2025-11-12 14:50:13 +01:00)


In [16]:
#processing the data
def process_csv(input_filepath, output_filepath, column_name):
    # Load the CSV file
    df = pd.read_csv(input_filepath)

    # Check if the specified column exists
    if column_name in df.columns:
        # Apply the check_text function to the specified column and create a new column with the results
        df['queries'] = df.apply(check_text, axis=1, args=(column_name,))
    else:
        print(f"Column '{column_name}' does not exist in the CSV file.")
        return

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output_filepath, index=False)
    print(f"Processed file saved as '{output_filepath}'.")

time: 0 ns (started: 2025-11-12 14:50:13 +01:00)


In [17]:
#processing the test data
input_csv = test_data_path
output_csv = 'processed_test_data.csv'
column_to_check = 'Instruction'
process_csv(input_csv, output_csv, column_to_check)

Processed file saved as 'processed_test_data.csv'.
time: 32 ms (started: 2025-11-12 14:50:13 +01:00)


In [18]:
#loading the processed test data
test_data_df=pd.read_csv('processed_test_data.csv')

time: 32 ms (started: 2025-11-12 14:50:13 +01:00)


In [19]:
#putting the data in some order and renaming the columns
test_data_df=test_data_df[['queries', 'Instruction', 'input_text']]
test_data_df.rename(columns={'Instruction': 'instructions', 'input_text': 'texts'}, inplace=True)

time: 0 ns (started: 2025-11-12 14:50:13 +01:00)


In [20]:
#verifying the ordered data
#test_data_df

time: 0 ns (started: 2025-11-12 14:50:13 +01:00)


In [21]:
#we need to generate a list of prompts will be passed to the model.
prompts=[]
for a,b,c in zip(test_data_df['queries'],test_data_df['instructions'],test_data_df['texts']):
    results=retrieve_vector_db(a)
    context=results[0]
    prompt = f'''
[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations. The output format should correspond to the one in the example sentences. Example sentences containing an instruction, input text and the extracted properties are given below:

Example Sentences: {context}

Instruction: {b}

Input Text: {c}

[/INST]
'''
    prompts.append(prompt)

time: 12.4 s (started: 2025-11-12 14:50:13 +01:00)


In [22]:
#verifying the list of prompts generated for each row in the dataset
#display the prompt
#prompts

time: 0 ns (started: 2025-11-12 14:50:25 +01:00)


In [23]:
#verify the length of the prompt list
print(len(prompts))

130
time: 16 ms (started: 2025-11-12 14:50:25 +01:00)


In [24]:
#GPT
import openai
import os
from IPython.display import Markdown
api_key="OPEN_API_KEY"

time: 375 ms (started: 2025-11-12 14:50:25 +01:00)


In [25]:
import openai

# Set your API key securely
openai.api_key = api_key

def chatWithGPT4(user_text, print_output=False):
    try:
        # Creating text completions using the updated `ChatCompletion` class
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",  # Easily switch between "gpt-3.5-turbo-0613", "gpt-4", "text-davinci-004", or others
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": user_text}],
            max_tokens=3000
        )
        text_output = response['choices'][0]['message']['content'] if response['choices'] else "No response generated."
        if print_output:
            print(text_output)
        return text_output
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

time: 15 ms (started: 2025-11-12 14:50:26 +01:00)


In [26]:
#checking a sample prompt at a particular index in the list of prompts
#prompts[0]

time: 0 ns (started: 2025-11-12 14:50:26 +01:00)


In [27]:
#getting the model responses in a list
responses=[]
for prompt in prompts:
    result=chatWithGPT4(prompt)
    responses.append(result)

time: 2min 29s (started: 2025-11-12 14:50:26 +01:00)


In [28]:
#dispalying the responses
responses

['Extracted Properties:\n\n3.3 THz',
 'Extracted Properties:\n\n10 mW',
 'Extracted Properties:\n\n2.07 THz, 2.02 THz',
 'Extracted Properties:\n\n190 K, 125 K',
 'Extracted Properties:\n\n107 K, 91 K',
 'Extracted Properties:\n\nGaAs/Al0.23Ga0.77As',
 'Extracted Properties:\n\nIn0.6Ga0.4As/InAs',
 'Extracted Properties:\n\n3.0 THz',
 'Extracted Properties:\n\n185 K',
 'Extracted Properties:\n\n325 K',
 'Extracted Properties:\n\nresonant-phonon',
 'Extracted Properties:\n\n195 K',
 'Extracted Properties:\n\n24 mW',
 'Extracted Properties:\n\nLO-phonon',
 'Extracted Properties:\n\n150 K, 110 K',
 'Extracted Properties:\n\nLO-phonon',
 'Extracted Properties:\n\n113 K, 89 K',
 'Extracted Properties:\n\n70K',
 'Extracted Properties:\n\nLO phonon',
 'Extracted Properties:\n\nresonant-phonon',
 'Extracted Properties:\n\n182 K',
 'Extracted Properties:\n\nnone',
 'Extracted Properties:\n\nGaAs/Al0.15Ga0.85As',
 'Extracted Properties:\n\nLO phonon',
 'Extracted Properties:\n\nLO-phonon-based d

time: 16 ms (started: 2025-11-12 14:52:55 +01:00)


In [29]:
#writing the model responses to a csv file
import csv

# Name of the csv file to save the list
file_name = "qcl-RAG-GPT4-Turbo-responses.csv"

# Writing the list to a csv file with one column
with open(file_name, "w", newline="") as file:
    writer = csv.writer(file)
    for item in responses:
        writer.writerow([item])

print("List saved to", file_name)

List saved to qcl-RAG-GPT4-Turbo-responses.csv
time: 0 ns (started: 2025-11-12 14:52:55 +01:00)
